In [15]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input, Embedding, Concatenate, Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Define the Generator
def build_generator(z_dim, img_shape, num_classes):
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.01))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.01))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.01))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    return model

# Define the Discriminator
def build_discriminator(img_shape, num_classes):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Define the GAN
def build_gan(generator, discriminator):
    z = Input(shape=(z_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Embedding(num_classes, z_dim)(label)
    label_embedding = Flatten()(label_embedding)
    joined_representation = Multiply()([z, label_embedding])
    img = generator(joined_representation)
    validity = discriminator([img, label])
    return Model([z, label], validity)

# Set parameters
z_dim = 100
img_shape = (224, 224, 1)  # Assuming images are grayscale with dimensions 224x224
num_classes = 7  # Assuming you have 7 classes in your dataset
data_folder = 'dataset/HAM10000_new_folder/'
csv_file = 'dataset/HAM10000_metadata.csv'
batch_size = 32
epochs = 10000
batch_count = 32
save_interval = 1000

# Build and compile the models
generator = build_generator(z_dim, img_shape, num_classes)
discriminator = build_discriminator(img_shape, num_classes)

discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])

discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

# Read the CSV file and convert labels to integers
df = pd.read_csv(csv_file)
label_dict = {label: idx for idx, label in enumerate(df['dx'].unique())}
df['label'] = df['dx'].map(label_dict)

# Training loop
for epoch in range(epochs):
    for _ in range(batch_count):
        noise = np.random.normal(0, 1, size=(batch_size, z_dim))
        sampled_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)

        generated_images = generator.predict([noise, sampled_labels])

        batch_idx = np.random.randint(0, df.shape[0], batch_size)
        real_images = []

        for idx in batch_idx:
            image_id = df.iloc[idx]['image_id']
            img_path = os.path.join(data_folder, f'{image_id}.jpg')
            img = load_img(img_path, color_mode='grayscale', target_size=(img_shape[0], img_shape[1]))
            img_array = img_to_array(img)
            img_array = img_array / 255.0
            real_images.append(img_array)

        real_images = np.array(real_images)
        labels = df.iloc[batch_idx]['label'].values.reshape(-1, 1)

        X = np.concatenate([real_images, generated_images])
        y_labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        y_labels = np.concatenate([y_labels, y_labels])

        d_loss = discriminator.train_on_batch([X, np.concatenate([labels, sampled_labels])], y_labels)

        noise = np.random.normal(0, 1, size=(batch_size, z_dim))
        sampled_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)

        g_loss = gan.train_on_batch([noise, sampled_labels], np.ones((batch_size, 1)))

    print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

    if epoch % save_interval == 0:
        # Save generated images here (if desired)
        pass

ValueError: in user code:

    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\User\AppData\Roaming\Python\Python311\s